# Text Klassifikation - Sentiment Analyse

In diesem Notebook möchten wir uns mit der Klassifikation von Texten beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von _Sentiment140_, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

---

Dieses Notebook gliedert sich in die folgenden Teile:

1. Datenanalyse und Preprocessing
2. Tokenisierung & Vokabular
3. Klassifikation
4. Klassifikation mit huggingface transformers

In [ ]:
import tensorflow as tf
tf.__version__

## 0. Vorbereitung

Als erstes muss der Datensatz von http://help.sentiment140.com/for-students in diese Colab Virtual Machine geladen werden.

In [ ]:
! gdown 0B04GJPshIjmPRnZManQwWEdTZjg

In [ ]:
DATA_DIR = '/content/sentiment140'

In [ ]:
!unzip /content/trainingandtestdata.zip -d {DATA_DIR}
%rm /content/trainingandtestdata.zip
%ls -la /{DATA_DIR}

## 1. Datenbeschaffung und -Analyse

Im ersten Schritt sollen folgende Schritte durchgeführt werden.

In [ ]:
import pandas as pd
import os

1. Ladet den Datensatz in einen Pandas Dataframe. Welche Feature gibt es? Wie viele Samples gibt es?

2. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte die Liste mit den Daten folgendes Format haben : `id => (polarity, text)`. 

3. Wandelt diese Werte in 1 (positiv) und 0 (negativ).

4. Fügt eine Spalte für die Anzahl an Wörtern im Text hinzu.

5. Analysiert den Datensatz mit den Pandas Boardmitteln.

In [ ]:
df = # TODO

In [ ]:
df['word_count'] = # TODO
print(f'Mean tweet length: X')

## 2. Tokenisierung

Um einen Einblick in die Daten zu bekommen und um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten nun aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten Sie die Tweets in Wörter aufteilen. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.

1. Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisiert damit die eingelesen Tweets. Am Ende solltet ihr eine Liste `tokenized = [(polarity, [token_1,token_2, ...])]` erhalten.

In [ ]:
df['tokenized'] = # TODO

Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegt euch, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser From zu behalten. Begründet kurz Ihre Entscheidungen.
Falls ihr euch entschlossen habt, nicht alle diese Bestandteile zu behalten, filtert dementsprechend eure Daten. Die Struktur Ihrer Daten sollte am Ende gleich bleiben: `df['cleaned'] = [token_1,...]`

In [ ]:
df['tokenized'] = # TODO

Zählt die Tokens in eurem Datensatz. Benutzt dafür ein Dictionary oder andere Python Build-Ins. 

1. Wieviele unterschiedliche Wörter gibt es?
2. Gebt die 100 häufigsten Wörter sortiert aus. 

Was zieht ihr aus den beiden Analysen? Was müsst ihr zusätzlich noch filtern?

In [ ]:
# TODO

Eine Möglichkeit für komplexere Preprocessing-Methoden ist das Entfernen von Stoppwörtern. Hiefür nutzen wir NLTK.

Filtert eure gesäuberten Tokens auf Stopwörter.

In [ ]:
import nltk

In [ ]:
df['cleaned'] = # TODO

## 3. Klassifikation

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Es handelt sich also um eine binäre Klassifikation.

Für das Training des neuronalen Netzes orintiert sich der nachfolgende Teil an _Keras_ als Framework. Ihr könnt aber auch ein anderes Framework wie bspw. _Pytorch_ benutzen. 

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, wir müssen unsere Daten aber noch in Vektoren transformieren.

Für die erste Klassifikation encoden wir die Eingabe als **Bag-of-Words**, sodass jedes potentielle Wort einem Eingabeneuron einspricht. _Ein Beispiel_: Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor [1,1,0,0], für den zweiten [0,0,2,1].

### 3.1 Vokabular

Befüllt das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`

In [ ]:
word2idx = # TODO

Welche Länge werden die Vektoren haben?

In [ ]:
VECTOR_LEN = # TODO

Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält. 
Bevor ihr damit beginnen, überschlagt, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht euer Hauptspeicher dafür aus?

In [ ]:
MEMORY = # TODO

### 3.2 Data Generator
Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigt die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:
 * d: tokenisierte und gesäuberte Tweets und Labels
 * w2i: das word2index dictionary
 * batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.
 
Die benutzen Tweets nacheinander aus `d` gewählt werden und kein Tweet mehrfach zurückgegeben werden.

In [ ]:
import random
import numpy as np

def data_generator(x, y, w2i, batch_size):
  num_batches = # TODO
  current_batch = 0

  while True:
    # Initialize with zeros
    batch_x = np.zeros((batch_size, len(w2i.keys())))
    batch_y = np.zeros((batch_size, 1))

    # TODO
  
    current_batch += 1
    yield batch_x, batch_y

    # Reset counter
    if current_batch >= num_batches:
      current_batch = 0

In [ ]:
gen = data_generator(...) # TODO

Ihr könnt euren Generator wie folgt ausprobieren:

In [ ]:
next(gen)

### 3.3 Neuronales Netz

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.

In [ ]:
from tensorflow.keras.models import Sequential
m = Sequential()

Fügt einen _Dense_-Layer dem Netz hinzu, als _hidden units_ könnt ihr 16 nehmen. Da dies auch der Eingabe-Layer ist, müsst ihr den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)

In [ ]:
m.add() # TODO

Als letzten Layer in unserem neuronalen Netz, fügt einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegt euch die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählt ihr?

In [ ]:
m.add() # TODO

7. Kompiliert das neuronale Netz. Als `optimizer` könnt ihr 'adam' benutzen. Wählt eine passende `loss`-Funktion aus. Begründet eure Entscheidung. (https://keras.io/models/model/#compile)

In [ ]:
# TODO

### 3.3. Training

Bevor ihr nun das neuronale Netz trainiert, teilt noch euren Datensatz in zwei Teile auf. Einen Teil zum Trainieren und einen zum Evaluieren. Das Verhältnis der beiden Datensätze sollte 70%:30% sein. Bevor ihr die Daten aufteilen, durchmischt sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem solltet ihr die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.

In [ ]:
# TODO

Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` könnt ihr 100 nehmen, für den `epochs`-Parameter 10. Was wählt ihr als `steps_per_epoch`-Parameter? (https://keras.io/models/model/#fit_generator)

In [ ]:
m.fit() # TODO
print(m.summary())

Während das Netz trainiert wird, könnt ihr euch Gedanken zur Evaluierung machen:
   * Definiert die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)
   * Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreibt diese Metrik und schreibt die Formel zur Berechnung auf.
   * Warum könnte die _Accuracy_ eine schlechte Metrik sein?
   * Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreibt diese Metriken mit eigenen Worten. Schreibt auch die Formeln zur Berechnung auf.
   * Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?

11. Inzwischen sollte das Netz fertig trainiert sein. Speichert es ab!

In [ ]:
m.save('my_net.h5')

### 3.4. Evaluation

Evaluiert euer Netz mit dem Datensatz, den ihr oben beseite gelegt haben. Benutzt dafür die `predict`-Methode des Models. Berechnet dafür _Precision_, _Recall_, _Accuracy_ und _Confusion Matrix_. 

Interpretiert kurz eure Ergebnisse. 

In [ ]:
# TODO

## 3b. (optional) Erweiterte Klassifikation
Im nächsten Schritt wollen wir das Netz auf dem vollen Datensatz trainieren und die Komplexität etwas erhöhen.

1. Trainiert euer Netz auf dem großen Datensatz.

In [ ]:
# TODO

2. Verändert die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainiert das Netz erneut (auf dem kleinen Datensatz). Was stellt ihr fest?

In [ ]:
# TODO

## 4. Sentiment Analyse mit Huggingface Transformers

In diesem Abschnitt werden wir nun Huggingface (https://huggingface.co/) zu Sentence Classification nutzen.

Hierfür benötigen wir die Libraries `datasets` (für preprocessing) und `transformers` (für die Transformer Modelle). Ebenfalls werden wir das Tokenisierung nicht mehr selbst durchführen, sondern viel mehr die `tokenizers` von huggingface hierfür benutzen.

Für eine gute Einführung in Klassifikation mit Transformers siehe: https://huggingface.co/docs/transformers/tasks/sequence_classification 

In [ ]:
!pip install datasets transformers

### 4.1 Data Preparation

Bevor wir ein huggingface transformer Modell nutzen können, müssen wir:

* den Datensatz (`df`) als `Dataset` laden
* ladet zunächst nur einen Teil des Datensatzes und wenn das Training funktioniert, vergrößert den Datensatz.
* die Spalten umbennenen 
* Train / Test Split (90 / 10)

1. Lade den Datensatz

In [ ]:
dataset = # TODO

2. Entferne alle unnötigen Spalten und benennen `polarity` in `labels` um. Achte darauf, dass die Spalte `labels` vom Feature-Typ `ClassLabel` ist.

In [ ]:
dataset = # TODO

In [ ]:
features = # TODO
dataset = dataset.cast(features)

3. Teilt den Datensatz in train und test split auf.

In [ ]:
dataset = # TODO

Schaut euch die Klassenverteilung im Datensatz an? Habt ihr noch eine 50/50 Verteilung zwischen positiv und negativ?

### 4.1. Training
Nun haben wir den Datensatz geladen und vorbereitet. 

1. Ladet ein `AutoModelForSequenceClassification` eurer Wahl (https://huggingface.co/models) und den entsprechenenden `AutoTokenizer`.

  - Welches Modell nutzt ihr? Auf welchen Daten wurde das Netz vortrainiert?
  - Auf welchem Framework basiert das Modell?

In [ ]:
tokenizer = # TODO

In [ ]:
model = # TODO

2. Bevor das eigentliche Training beginnt, muss der Datensatz noch tokenized werden. Nutzt hierfür den oben instantiierten `tokenizer` und beantwortet folgende Fragen:
  - Wie groß ist das des Tokenizers? 
  - In welcher Granularität werden Texte aufgesplittet?

In [ ]:
tokenized_dataset = # TODO

3. Baut euch eine Methode `compute_metrics`, die die notwendigen Metriken während des Trainings sammelt. Nutzt hierfür wieder Precision, Recall und F1 score.

In [ ]:
def compute_metrics(eval_pred):
  """ Computes the metrics given a tuple of (logits, labels) """
  
  # TODO 
  return {
    "accuracy": accuracy,
    "f1": f1,
    "precision": precision,
    "recall": recall,
  }

4. Ladet einen `Trainer` und spezifiziert die `TrainingArguments`. Es bietet sich an, nach jeder Epoche zu evaluieren und das beste Modell am Ende zu laden.

In [ ]:
finetuning_args = TrainingArgument() # TODO
trainer = Trainer() # TODO

5. Trainiert das Modell und behaltet die Metriken im Auge.

In [ ]:
trainer.train()

### 4.3. (Optional) Training mit weiterem Modell

Falls ihr möchtet, könnt ihr optional das geladene vortrainierte Modell austauschen und ein anderes (größeres, auf anderen Daten trainiert, ...) Modell nutzen.


### 4.4. Prediction

Nun könnt ihr das Modell nutzen, um ein paar Beispiel aus dem Test-Datensatz zu predicten.

In [ ]:
id2label = { 0: "neg", 1: "pos" }
prediction_ds = tokenized_dataset['test'].select(range(50))

Gebt die falschen Predictionn aus. Was fällt euch auf?

In [ ]:
raw_pred, _, _ = trainer.predict(prediction_ds)
print(f"Total predictions: {len(raw_pred)}")

for text, prediction in zip(prediction_ds, np.argmax(raw_pred, axis=1)):
    expected = text['labels']
    if expected != prediction:
      print(f"Text: {text['text']} --- Expected: {id2label[expected]} --- Predicted: {id2label[prediction]}")